In [11]:
import gc
import math
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
import torch
import torch.cuda
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
from tqdm import tqdm
import torchvision.models as models
from torchvision.transforms import transforms as T
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
import time
from datetime import datetime
from torchtext.data import Field, BucketIterator
import re
from spacy.tokenizer import Tokenizer
from collections import defaultdict, Counter
from typing import List

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataset Loading

In [13]:
path = '../PHOENIX-2014-T-release-v3/PHOENIX-2014-T/annotations/manual/PHOENIX-2014-T.train-complex-annotation.corpus.csv'
dataframe = pd.read_csv(path, sep='|')
print (dataframe.head(5))
# Removing start and end columns
dataframe.drop(columns=['start', 'end'], inplace=True)

# Working on signer 1 alone
signer1_dataframe = dataframe[dataframe['speaker']=='Signer01']

signer1_dataframe.shape

                                   name  \
0  11August_2010_Wednesday_tagesschau-1   
1  11August_2010_Wednesday_tagesschau-4   
2  11August_2010_Wednesday_tagesschau-5   
3  11August_2010_Wednesday_tagesschau-6   
4  11August_2010_Wednesday_tagesschau-7   

                                          video  start  end   speaker  \
0  11August_2010_Wednesday_tagesschau-1/1/*.png     -1   -1  Signer08   
1  11August_2010_Wednesday_tagesschau-4/1/*.png     -1   -1  Signer08   
2  11August_2010_Wednesday_tagesschau-5/1/*.png     -1   -1  Signer08   
3  11August_2010_Wednesday_tagesschau-6/1/*.png     -1   -1  Signer08   
4  11August_2010_Wednesday_tagesschau-7/1/*.png     -1   -1  Signer08   

                                                orth  \
0  __ON__ JETZT WETTER MORGEN DONNERSTAG ZWOELF F...   
1  ORT-PLUSPLUS REGEN DURCH REGEN-PLUSPLUS KOENNE...   
2  __ON__ loc-NORDWEST HEUTE NACHT TROCKEN BLEIBE...   
3  TAGSUEBER OFT REGEN-PLUSPLUS GEWITTER GEWITTER...   
4  __ON__ WOLKE LOCH L

(1862, 5)

## Analysing Dataset

In [4]:
if False:
    m = 0
    max_folder = ''
    list_folders = []
    list_len = []

    for folder in os.listdir('../PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/train/'):
        l = len(os.listdir('../PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/train/' + folder))
        list_folders.append(folder)
        list_len.append(l)

    temp_df = pd.DataFrame(list(zip(list_folders, list_len)), columns=['Folder', 'No. of images'])
    
    plt.hist(temp_df['No. of images'], bins=100)
    plt.show()

## Tokenizing

In [37]:
# loaded_data = np.array(signer1_dataframe['orth'])
# for num in range (len(loaded_data)):
#     sentence = loaded_data[num].split()
#     sentence.insert(0, '<sos>')
#     space = " "
#     sentence = space.join(sentence)
#     loaded_data[num] = sentence

In [6]:
from torchnlp.encoders.text import StaticTokenizerEncoder, SpacyEncoder, pad_tensor
loaded_data = np.array(signer1_dataframe['orth'])
encoder = StaticTokenizerEncoder(loaded_data, tokenize=lambda s: s.split(), append_eos=True)

encoded_data = [encoder.encode(example) for example in loaded_data]
encoded_data = [pad_tensor(x, length=35) for x in encoded_data]

example_encode = encoder.encode(loaded_data[1])
example_pad = pad_tensor(example_encode, length=35)

print('actual: ', loaded_data[1])
print('encoded: ',example_pad)
print('decoded: ', encoder.decode(example_pad))

actual:  __ON__ ITALIEN IX TIEF DRUCK cl-KOMMEN HEUTE NACHT BERG SCHNEE REGEN REGEN __OFF__
encoded:  tensor([ 5, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 24, 14,  2,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
decoded:  __ON__ ITALIEN IX TIEF DRUCK cl-KOMMEN HEUTE NACHT BERG SCHNEE REGEN REGEN __OFF__ </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>


## Alternate Tokenizer

In [10]:
SIL_TOKEN = "<si>"
UNK_TOKEN = "<unk>"
PAD_TOKEN = "<pad>"
BOS_TOKEN = "<s>"
EOS_TOKEN = "</s>"

class Vocabulary:
    """ Vocabulary represents mapping between tokens and indices. """

    def __init__(self):
        # don't rename stoi and itos since needed for torchtext
        # warning: stoi grows with unknown tokens, don't use for saving or size
        self.specials = []
        self.itos = []
        self.stoi = None
        self.DEFAULT_UNK_ID = None

    def _from_list(self, tokens: List[str] = None):
        """
        Make vocabulary from list of tokens.
        Tokens are assumed to be unique and pre-selected.
        Special symbols are added if not in list.
        :param tokens: list of tokens
        """
        self.add_tokens(tokens=self.specials + tokens)
        assert len(self.stoi) == len(self.itos)

    def _from_file(self, file: str):
        """
        Make vocabulary from contents of file.
        File format: token with index i is in line i.
        :param file: path to file where the vocabulary is loaded from
        """
        tokens = []
        with open(file, "r", encoding="utf-8") as open_file:
            for line in open_file:
                tokens.append(line.strip("\n"))
        self._from_list(tokens)

    def __str__(self) -> str:
        return self.stoi.__str__()

    def to_file(self, file: str):
        """
        Save the vocabulary to a file, by writing token with index i in line i.
        :param file: path to file where the vocabulary is written
        """
        with open(file, "w", encoding="utf-8") as open_file:
            for t in self.itos:
                open_file.write("{}\n".format(t))

    def add_tokens(self, tokens: List[str]):
        """
        Add list of tokens to vocabulary
        :param tokens: list of tokens to add to the vocabulary
        """
        for t in tokens:
            new_index = len(self.itos)
            # add to vocab if not already there
            if t not in self.itos:
                self.itos.append(t)
                self.stoi[t] = new_index

    def is_unk(self, token: str) -> bool:
        """
        Check whether a token is covered by the vocabulary
        :param token:
        :return: True if covered, False otherwise
        """
        return self.stoi[token] == self.DEFAULT_UNK_ID()

    def __len__(self) -> int:
        return len(self.itos)
    
    

class GlossVocabulary(Vocabulary):
    def __init__(self, tokens: List[str] = None, file: str = None):
        """
        Create vocabulary from list of tokens or file.
        Special tokens are added if not already in file or list.
        File format: token with index i is in line i.
        :param tokens: list of tokens
        :param file: file to load vocabulary from
        """
        super().__init__()
        self.specials = [SIL_TOKEN, UNK_TOKEN, PAD_TOKEN]
        self.DEFAULT_UNK_ID = lambda: 1
        self.stoi = defaultdict(self.DEFAULT_UNK_ID)

        if tokens is not None:
            self._from_list(tokens)
        elif file is not None:
            self._from_file(file)

        # TODO (Cihan): This bit is hardcoded so that the silence token
        #   is the first label to be able to do CTC calculations (decoding etc.)
        #   Might fix in the future.
        assert self.stoi[SIL_TOKEN] == 0
        
    def array_to_sentence(self, array: np.array, cut_at_eos=True) -> List[str]:
        """
        Converts an array of IDs to a sentence, optionally cutting the result
        off at the end-of-sequence token.
        :param array: 1D array containing indices
        :param cut_at_eos: cut the decoded sentences at the first <eos>
        :return: list of strings (tokens)
        """
        sentence = []
        for i in array:
            s = self.itos[i]
            if cut_at_eos and s == EOS_TOKEN:
                break
            sentence.append(s)
        return sentence

    def arrays_to_sentences(self, arrays: np.array) -> List[List[str]]:
        gloss_sequences = []
        for array in arrays:
            sequence = []
            for i in array:
                sequence.append(self.itos[i])
            gloss_sequences.append(sequence)
        return gloss_sequences


def filter_min(counter: Counter, minimum_freq: int):
    """ Filter counter by min frequency """
    filtered_counter = Counter({t: c for t, c in counter.items() if c >= minimum_freq})
    return filtered_counter


def sort_and_cut(counter: Counter, limit: int):
    """ Cut counter to most frequent,
    sorted numerically and alphabetically"""
    # sort by frequency, then alphabetically
    tokens_and_frequencies = sorted(counter.items(), key=lambda tup: tup[0])
    tokens_and_frequencies.sort(key=lambda tup: tup[1], reverse=True)
    vocab_tokens = [i[0] for i in tokens_and_frequencies[:limit]]
    return vocab_tokens

def build_vocab(
    field: str, max_size: int, min_freq: int, dataset: Dataset, vocab_file: str = None
) -> Vocabulary:
    """
    Builds vocabulary for a torchtext `field` from given`dataset` or
    `vocab_file`.
    :param field: attribute e.g. "src"
    :param max_size: maximum size of vocabulary
    :param min_freq: minimum frequency for an item to be included
    :param dataset: dataset to load data for field from
    :param vocab_file: file to store the vocabulary,
        if not None, load vocabulary from here
    :return: Vocabulary created from either `dataset` or `vocab_file`
    """

    if vocab_file is not None:
        # load it from file
        if field == "gls":
            vocab = GlossVocabulary(file=vocab_file)
        elif field == "txt":
            vocab = TextVocabulary(file=vocab_file)
        else:
            raise ValueError("Unknown vocabulary type")
    else:
        tokens = []
        for i in dataset.examples:
            if field == "gls":
                tokens.extend(i.gls)
            elif field == "txt":
                tokens.extend(i.txt)
            else:
                raise ValueError("Unknown field type")

        counter = Counter(tokens)
        if min_freq > -1:
            counter = filter_min(counter, min_freq)
        vocab_tokens = sort_and_cut(counter, max_size)
        assert len(vocab_tokens) <= max_size

        if field == "gls":
            vocab = GlossVocabulary(tokens=vocab_tokens)
        elif field == "txt":
            vocab = TextVocabulary(tokens=vocab_tokens)
        else:
            raise ValueError("Unknown vocabulary type")

        assert len(vocab) <= max_size + len(vocab.specials)
        assert vocab.itos[vocab.DEFAULT_UNK_ID()] == UNK_TOKEN

    for i, s in enumerate(vocab.specials):
        if i != vocab.DEFAULT_UNK_ID():
            assert not vocab.is_unk(s)

    return vocab

def tokenize_text(text):
    return text.split()


gls_field = data.Field(
        pad_token=PAD_TOKEN,
        tokenize=tokenize_text,
        batch_first=True,
        lower=False,
        include_lengths=True)

gls_vocab = build_vocab(
        field="gls",
        min_freq=1,
        max_size=1000,
        dataset=loaded_data)

NameError: name 'List' is not defined

In [27]:
# custom spacy tokenizer

# special_cases = {":)": [{"ORTH": ":)"}]}
prefix_re = re.compile(r'''^[[("']''')
suffix_re = re.compile(r'''[])"']$''')
infix_re = re.compile(r'''[-~]''')
simple_url_re = re.compile(r'''^https?://''')

def custom_tokenizer(nlp):
    return Tokenizer(nlp.vocab, rules=special_cases,
                                prefix_search=prefix_re.search,
                                suffix_search=suffix_re.search,
                                infix_finditer=infix_re.finditer,
                                url_match=simple_url_re.match)

/miniconda3/envs/transformers/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Possible nested set at position 2
  """


In [7]:
# spacy_ger = spacy.load("de")

german = Field(lower=True, init_token="<sos>", eos_token="<eos>", fix_length = 50)

In [8]:
loaded_data = np.array(signer1_dataframe['orth'])
german.build_vocab(loaded_data, max_size=10000, min_freq=1)

In [9]:
german.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7fafcdf73950>>,
            {'<unk>': 0,
             '<pad>': 1,
             '<sos>': 2,
             '<eos>': 3,
             ' ': 4,
             'E': 5,
             'N': 6,
             '_': 7,
             'O': 8,
             'S': 9,
             'R': 10,
             'I': 11,
             'T': 12,
             'A': 13,
             'H': 14,
             'G': 15,
             'U': 16,
             'D': 17,
             'M': 18,
             'F': 19,
             'L': 20,
             'C': 21,
             'W': 22,
             'K': 23,
             'B': 24,
             '-': 25,
             'P': 26,
             'Z': 27,
             'l': 28,
             'c': 29,
             'o': 30,
             'V': 31,
             'X': 32,
             's': 33,
             'J': 34,
             'p': 35,
             '?': 36,
             'e': 37,
             'g': 38,
             'n': 39,
         

## Transforms

In [ ]:
normalize = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transform = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor(),normalize])

## Filtering Sequences

In [ ]:
x = []
signer1_path = '../PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/train/'
signer1_main = signer1_dataframe.copy(deep=True)
for folder in tqdm(signer1_main['name']):
    sequence_length = len(os.listdir(signer1_path + folder))
    
    if sequence_length > 250 or sequence_length < 50:
        signer1_dataframe = signer1_dataframe[signer1_dataframe['name']!=folder]

signer1_train, signer1_test = train_test_split(signer1_dataframe, test_size=0.3, random_state=42)
signer1_test, signer1_val = train_test_split(signer1_test, test_size=0.5, random_state=42)

signer1_train.shape, signer1_test.shape, signer1_val.shape

## Dataloader

In [ ]:
class SLRT_Signer(Dataset):
    """SLRT dataset."""

    def __init__(self, data_frame, root_dir, transform, tokenizer):
        self.images_frame = data_frame['name']
        self.glosses = data_frame['orth']
        self.root_dir = root_dir
        self.transform = transform
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.images_frame)

    def __getitem__(self, idx):
        global device
        if torch.is_tensor(idx):
            idx = idx.tolist()
        training_example = torch.zeros(250, 3, 224, 224)
        for files in os.listdir(os.path.join(self.root_dir, self.images_frame.iloc[idx])):
            img_name = self.root_dir + self.images_frame.iloc[idx] + '/' + files
            image = Image.open(img_name)
            image = self.transform(image)
            training_example[0:len(files), :] = image

        gloss = self.glosses.iloc[idx]
        encoded_gloss = self.tokenizer.encode(gloss)
        encoded_gloss = pad_tensor(encoded_gloss, 250)
        
        return training_example, encoded_gloss


In [ ]:
signer1_train_dataset = SLRT_Signer(signer1_train,
                   root_dir='../PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/train/',
                   transform=transform,
                   tokenizer=encoder
                   )
                  
signer1_test_dataset = SLRT_Signer(signer1_test,
                   root_dir='../PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/train/',
                   transform=transform,
                   tokenizer=encoder
                   )

signer1_val_dataset = SLRT_Signer(signer1_val,
                   root_dir='../PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/train/',
                   transform=transform,
                   tokenizer=encoder
                  )

In [ ]:
params = {
    'batch_size': 12,
    'shuffle': True,
    'num_workers': 0
}
max_epochs = 100

In [ ]:
train_gen = DataLoader(signer1_train_dataset, **params)
test_gen = DataLoader(signer1_test_dataset, **params)
val_gen = DataLoader(signer1_val_dataset, **params)

## Squeeze Net

In [ ]:
class SqueezeNet(nn.Module):
    
    def __init__(self, pre_train=False):
        super(SqueezeNet, self).__init__()
        
        self.model = torch.hub.load('pytorch/vision:v0.6.0', 'squeezenet1_1', pretrained=True)
        
        if pre_train == True:
            for param in self.model.parameters():
                param.requires_grad = False
            self.model.eval()
        
        children = [child for child in self.model.children()]    
        for child in children[0][:-5]:
            for param in child.parameters():
                param.requires_grad = False
        
                
        self.fc1 = nn.Linear(1000, 512)
        
    def forward(self, src):
        
        output = self.model(src)
        output = self.fc1(output)
        output = F.relu(output)
#         output = output.to(torch.device('cpu'))
        return output
        


In [ ]:
# ### Testing Squeezenet

# path=r'../PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/train/01April_2010_Thursday_heute-6694/images0010.png'

# squeeze_model = SqueezeNet().to(device)
# summary(squeeze_model, (3, 224, 224))

In [ ]:
# image = Image.open(path)
# image = transform(image)
# image = image.unsqueeze(0).to(device)


# output = squeeze_model(image)
# output.shape

## Positional Encoding

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

## Transformer User Defined

In [ ]:
# class TransformerModel(nn.Module):

#     def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
#         super(TransformerModel, self).__init__()
#         self.model_type = 'Transformer'
#         self.src_mask = None
#         self.tgt_mask = None
#         self.nopeakmask = None
#         self.pos_encoder = PositionalEncoding(ninp, dropout).to(device)
#         encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
#         decoder_layers = TransformerDecoderLayer(ninp, nhead, nhid, dropout)
#         self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
#         self.transformer_decoder = TransformerDecoder(decoder_layers, nlayers)
#         #self.encoder = nn.Embedding(ntoken, ninp)
#         self.decoder_embedding = nn.Embedding(ntoken, ninp)
#         self.ninp = ninp
#         self.decoder = nn.Linear(ninp, ntoken)
#         #self.softmax = nn.Softmax(1)
        
#         #self.init_weights()

#     def _generate_square_subsequent_mask(self, src, trt, sz):
#         mask = (torch.triu(torch.ones(250, 250)) == 1).transpose(0, 1).half().to(device)
#         nopeakmask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0)).half().to(device)
#         zeros = torch.zeros(self.ninp).half().to(device)
#         src_msk = (src == zeros).half().to(device)
#         target_msk = (trt == 0).unsqueeze(0).half().to(device)
#         return src_msk, target_msk, nopeakmask

#     def init_weights(self):
#         initrange = 0.1
#         self.decoder.bias.data.zero_()
#         self.decoder.weight.data.uniform_(-initrange, initrange)

#     def forward(self, src, trt):
# #         trt = trt.to(torch.device('cuda'))
#         trt = trt.to(src.device)
#         if self.src_mask is None or self.src_mask.size(0) != src.size(0):
#             device = src.device
#             src_mask, tgt_mask, self.nopeakmask = self._generate_square_subsequent_mask(src, trt, src.size(1))
#             self.src_mask = src_mask
#             #self.tgt_mask = tgt_mask
#         #src = self.encoder(src) * math.sqrt(self.ninp)
#         src = self.pos_encoder(src)
# #         print ("source",src.shape)
#         src = src.permute(1,0,2)
# #         print ("source",src.shape)
#         output = self.transformer_encoder(src)
# #         trt = trt.to(torch.cuda.device('cpu'))
# #         trt = torch.cuda.LongTensor(trt)
#         trt = trt.type(torch.cuda.LongTensor)
        
#         trgt = self.decoder_embedding(trt)
#         trgt = self.pos_encoder(trgt)
#         trgt = trgt.permute(1,0,2)
# #         trgt = trgt[:-1, :]
# #         print (trgt.shape), print (output.shape)
#         output = self.transformer_decoder(trgt, output, tgt_mask = self.nopeakmask) #tgt_key_padding_mask = tgt_mask)
#         output = self.decoder(output)
# #         print (output.shape)  shape: 250, batchsize, 719
#         output = output.permute(1,2,0)
#         #output = output.reshape(-1, output.shape[2])
# #         output = self.softmax(output)
#         return output

## Transformer Orginal Class

In [ ]:
class TransformerModelOriginal(nn.Module):
    def __init__(
        self, embedding_size, src_vocab_size, trg_vocab_size, src_pad_idx, num_heads, num_encoder_layers, num_decoder_layers,
        forward_expansion, dropout, max_len, device):
        
        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)
        
        self.device = device
        self.transformer = nn.Transformer(embedding_size, num_heads, num_encoder_layers, num_decoder_layers, forward_expansion, dropout)
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, src):
        src_mask = src.transpose(0, 1) == self.src_pad_idx

        # (N, src_len)
        return src_mask.to(self.device)

    def forward(self, src, trg):
        src_seq_length, N = src.shape
        trg_seq_length, N = trg.shape
        
#         src_positions = (torch.arange(0, src_seq_length).unsqueeze(1).expand(src_seq_length, N).to(self.device))
        
        trg_positions = (torch.arange(0, trg_seq_length).unsqueeze(1).expand(trg_seq_length, N).to(self.device))

#         embed_src = self.dropout((self.src_word_embedding(src) + self.src_position_embedding(src_positions)))
        
        embed_trg = self.dropout((self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions)))

        src_padding_mask = self.make_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(self.device)

        out = self.transformer(embed_src, embed_trg, src_key_padding_mask=src_padding_mask,tgt_mask=trg_mask)
        out = self.fc_out(out)
        return out

## Parameters

In [ ]:
ntokens = len(encoder.vocab) # the size of vocabulary
emsize = 512 # embedding dimension
nhid = 1024 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 4 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 4 # the number of heads in the multiheadattention models
dropout = 0.3 # the dropout value
src_pad_index = 0

## Sign2Gloss Class

In [ ]:
class Sign2Gloss_Model(nn.Module):
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super (Sign2Gloss_Model, self).__init__()
        self.TransformerModel = TransformerModel(ntokens, ninp, nhead, nhid, nlayers, \
                                                 dropout).half()
        self.SqueezeNet = SqueezeNet(pre_train=False).half().to(device)
        self.softmax = nn.Softmax(2)
        self.ninp = ninp
    
    def forward(self, src, trt):
        batch_vector = torch.Tensor(src.shape[0], src.shape[1], self.ninp).half().to(device)
        batch_batch_size = 32
        for i in range (src.shape[0]):
            for batch in range (0, 250, batch_batch_size):
                inp = src[i, batch:batch+batch_batch_size].half().to(device)
                transformer_source = self.SqueezeNet(inp)
                batch_vector[i, batch:batch+batch_batch_size]=transformer_source
                del transformer_source, inp
                gc.collect
#             batch_vector[i] = transformer_source 
#         batch_vector = batch_vector.half().to(device)
        self.TransformerModel = self.TransformerModel.to(device)
        output = self.TransformerModel(batch_vector, trt)
        output = self.softmax(output)
        del batch_vector
        
        return output

In [ ]:
# eval_model  = TransformerModel(ntoken=ntokens, ninp=emsize, nhid=nhid, nhead=nhead, nlayers=nlayers, dropout=dropout).to(device)

# summary(eval_model, [(-1,250, 512), (8, 250,)])

## Training Loop

In [ ]:
# #model training
# model = Sign2Gloss_Model(ntoken = ntokens, ninp = emsize, nhid = nhid, nlayers = nlayers, nhead = nhead, dropout = dropout)
# criterion = nn.CrossEntropyLoss(ignore_index = src_pad_index)
# optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#  # enumerate epochs
# for epoch in tqdm(range(30)):
# # enumerate mini batches
#     epoch_loss = 0
#     btch=1
# #     print ("Epoch: ", epoch, "in progress...")
#     for (inputs, targets) in train_gen:
#         print (btch, end=' ')
# #         inputs, targets = inputs, targets
#         # clear the gradients
#         optimizer.zero_grad()
#         # compute the model output
#         yhat = model(inputs, targets)
# #         model = model.to(device)
#         yhat = yhat.to(torch.device('cpu'))
#         yhat = yhat.type(torch.Tensor)
#         targets = targets.type(torch.LongTensor)
#         # calculate loss
#         loss = criterion(yhat, targets)
#         # credit assignment
#         loss.backward()
#         # update model weights
#         optimizer.step()
# #         model = model.to(torch.device('cpu'))
#         epoch_loss += loss.item()
#         torch.cuda.empty_cache()
#         del inputs, targets
#         gc.collect()
#         btch+=1
    
#     torch.save({
#     'epoch': epoch,
#     'model_state_dict': model.state_dict(),
#     'optimizer_state_dict': optimizer.state_dict(),
#     'loss': 100
#     }, 'squeezenetmode2.pt')
    
#     try:
#         with open('SqueezeNet.txt', 'at') as file:
#             now = datetime.now()
#             current_time = now.strftime("%H:%M:%S")
#             file.write("Epoch {}, Logs:{}, Time: {}\n".format(epoch, epoch_loss, current_time))
#     except:
#         pass

#     print ("Epoch {} : {}".format(epoch, epoch_loss))

In [ ]:
# import torch
# import gc
# for obj in gc.get_objects():
#     try:
#         if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
#             print(type(obj), obj.size())
#     except:
#         pass


In [ ]:
# torch.save({
#             'epoch': 10,
#             'model_state_dict': model.state_dict(),
#             'optimizer_state_dict': optimizer.state_dict(),
#             'loss': 100
#             }, 'squeezenetmodel-2_backup.pt')